---

In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os

In [45]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 200)

In [3]:
team_id = '81b62c00-e2a3-490d-84b9-d239aa567389'

In [4]:
token = 'e3f62371-d693-45fa-b741-f485300c0d9d'

In [5]:
host = 'https://api.natlogger.com'

In [6]:
url_parameter = '/api/parameter'
url_device = '/api/device'
url_measure = '/api/measure'
url_circle = '/api/measure/circle'
url_csv = '/api/measure/csv'

In [58]:
class Natlogger(object):
    def __init__(self, 
                 token, 
                 team_id,
                 host,
                 url_parameter, 
                 url_device, 
                 url_measure,
                 url_circle,
                 url_csv):
        self.token = token
        self.team_id = team_id
        self.host = host
        self.url_parameter = url_parameter
        self.url_device = url_device
        self.url_measure = url_measure
        self.url_circle = url_circle
        self.url_csv = url_csv
        self.header = {'token' : token}
        
    def list_of_params(self, count=1000):
        self.count = count
        params = {'teamId': self.team_id,
                  'calculate': 'true',
                  'count': count,
                  'enrich':'true'}
        response = requests.get(url=self.host+self.url_parameter, headers=self.header, params=params)
        if response:
            print('OK')
        else:
            print('Failed')
        df_params_list = pd.DataFrame(response.json()['data'])
        parameters_names_dict = dict(zip(df_params_list.name, df_params_list.id))
        return df_params_list, parameters_names_dict

    def list_of_devices(self, count=1000):
        self.count = count
        params = {'teamId': self.team_id,
                  'calculate': 'true',
                  'count': 35,
                  'enrich':'true'}
        response = requests.get(url=self.host+self.url_device, headers=self.header, params=params)
        
        if response:
            print('OK')
        else:
            print('Failed')
            
        df_devices = pd.DataFrame(response.json()['data'])
        devices_names_dict = dict(zip(df_devices.name, df_devices.id))
        return df_devices, devices_names_dict
    
    def get_data(self, 
                 required_device_name, 
                 devices_names_dict, 
                 required_parameters_names,
                 parameters_names_dict,
                 start_date=None,
                 end_date=None,
                 group_type='AUTO'):
        
        self.required_device_name = required_device_name
        self.devices_names_dict = devices_names_dict
        self.required_parameters_names = required_parameters_names
        self.parameters_names_dict = parameters_names_dict
        self.start_date = start_date
        self.end_date = end_date
        self.group_type = group_type
        
        device_id = devices_names_dict[required_device_name]
        
        parameters_ids = []
        for parameter in required_parameters_names:
            parameters_ids.append(parameters_names_dict[parameter])
        parameters_ids = ','.join(parameters_ids)
        
        params = {'teamId': team_id,
                  'calculate': 'true',
                  'deviceIds[]': device_id,
                  'parameterIds[]': parameters_ids,
                  'startDate': start_date.strftime('%Y-%m-%dT%H:%M:%S.%fZ'),
                  'endDate': end_date.strftime('%Y-%m-%dT%H:%M:%S.%fZ'),
                  'groupType': group_type,
                  'realtime': 'false'}
        if start_date and end_date == None:
            print('get today data')
        else:
            print('get data from ', 
                  start_date.strftime('%Y-%m-%dT%H:%M:%S.%fZ'), 
                  ' to ', 
                  end_date.strftime('%Y-%m-%dT%H:%M:%S.%fZ'))
        print('grouping to ', group_type)
        print('device: ', required_device_name)
        print('required params: ', required_parameters_names)
        
        response = requests.get(url=host+url_measure, headers=self.header, params=params)
        
        if response:
            print('OK')
        else:
            print('Failed')
        df_data = pd.DataFrame(response.json())
        #df_data['date'] = pd.to_datetime(df_data['date'], utc=True) + datetime.timedelta(hours=3)
        
        frames = []
        for i in range(len(df_data)):
            test_dict = df_data['measures'][i][devices_names_dict[required_device_name]]['parameterValues']
            dict3 = dict()
            for key, values in parameters_names_dict.items():
                try:
                    dict3[key] = test_dict[values]
                except:
                    pass
            frames.append(pd.DataFrame(data=dict3, index=[df_data['date'][i]]))
        if frames != []:
            df_all = pd.concat(frames)
            self.df_all = df_all
            return df_all
        else:
            print('Empty data')
            return []
        
             
    def plot_params(self, df):
        self.df = df
        for param in df.columns:
            if param == 'date':
                pass
            else:
                plt.figure(figsize=(16,8))
                plt.grid(True)
                sns.scatterplot(y=df[param], x=df.index).set(xlabel='Date and time', ylabel=param+', mg/m3')
                plt.show()
                
    def save_records_to_excel(self):
        df_all = self.df_all
        df_all = df_all.reset_index()
        df_all = df_all.rename(columns={'index': 'date'})
        df_all['date'] = pd.to_datetime(df_all['date'])
        df_all['date'] = df_all['date'].dt.tz_localize(None)
        df_all.to_excel(self.required_device_name + '.xlsx', index=False)
        
    def save_plots(self):
        for param in self.df.columns:
            if param == 'date':
                pass
            else:
                plt.figure(figsize=(16,8))
                plt.grid(True)
                sns.scatterplot(y=df[param], x=df.index).set(xlabel='Date and time', ylabel=param + ', mg/m3')
                plt.savefig(datetime.datetime.now().strftime('%Y-%m-%d') + '/' + param + '_' + self.required_device_name + '.png', dpi=300)
                plt.close()

In [59]:
natlogger = Natlogger(token, 
                      team_id, 
                      host, 
                      url_parameter, 
                      url_device, 
                      url_measure, 
                      url_circle, 
                      url_csv)

In [60]:
df_params, parameters_names_dict = natlogger.list_of_params()

OK


In [61]:
df_params

,id,created,changed,code,name,global,description,unitId,unitName,deleted,teamId,teamName,attachment,circle,concentration,sknIdentifier,unitNameLocalized
0,ea53f5a6-a696-4fe3-9623-8f0e2be7f8dd,2023-02-01T05:18:06.030Z,2023-02-01T05:18:06.030Z,iceMass,1;2;iceMass,False,None,c58c057a-d138-496f-be27-7259fba17ef6,"i18n{""en"": ""Kilogram"", ""ru"": ""Килограмм"", ""es""...",False,81b62c00-e2a3-490d-84b9-d239aa567389,SKN&Meteo_Aikhal_test,False,False,False,4.294967e+09,Килограмм
1,a5204eed-5c99-4cfd-8f78-6d25cd9dffaf,2023-02-01T05:18:06.028Z,2023-02-01T05:18:06.028Z,wireLength,1;2;wireLength,False,None,c99bb5e8-c7b3-41d0-a6e8-4427c72e375e,"i18n{""en"": ""meter"", ""ru"": ""Метр"", ""es"": ""meter""}",False,81b62c00-e2a3-490d-84b9-d239aa567389,SKN&Meteo_Aikhal_test,False,False,False,4.294967e+09,Метр
2,a1485b10-1bda-4aae-8755-8971e6033480,2023-02-01T05:18:06.025Z,2023-02-01T05:18:06.025Z,mechStress,1;2;mechStress,False,None,1f6fc108-1978-485b-9b27-6fc440b3e089,"i18n{""en"": ""V"", ""ru"": ""В"", ""es"": ""V""}",False,81b62c00-e2a3-490d-84b9-d239aa567389,SKN&Meteo_Aikhal_test,False,False,False,4.294967e+09,В
3,b4d0ff0f-af24-494e-8342-e85167025a22,2023-02-01T05:18:06.024Z,2023-02-01T05:18:06.024Z,wireSag,1;2;wireSag,False,None,c99bb5e8-c7b3-41d0-a6e8-4427c72e375e,"i18n{""en"": ""meter"", ""ru"": ""Метр"", ""es"": ""meter""}",False,81b62c00-e2a3-490d-84b9-d239aa567389,SKN&Meteo_Aikhal_test,False,False,False,4.294967e+09,Метр
4,f46c8014-52c5-4205-8a46-9d5db5e19cb6,2023-02-01T05:18:06.023Z,2023-02-01T05:18:06.023Z,ctState,1;2;ctState,False,None,1f6fc108-1978-485b-9b27-6fc440b3e103,"i18n{""en"": ""State"", ""ru"": ""Состояние"", ""es"": ""...",False,81b62c00-e2a3-490d-84b9-d239aa567389,SKN&Meteo_Aikhal_test,False,False,False,4.294967e+09,Состояние
5,a5034d65-1284-4af9-8456-a1c74f533970,2023-02-01T05:18:06.022Z,2023-02-01T05:18:06.022Z,batteryVoltage,1;2;batteryVoltage,False,None,1f6fc108-1978-485b-9b27-6fc440b3e089,"i18n{""en"": ""V"", ""ru"": ""В"", ""es"": ""V""}",False,81b62c00-e2a3-490d-84b9-d239aa567389,SKN&Meteo_Aikhal_test,False,False,False,4.294967e+09,В
6,d5b2667d-f093-42fe-b9cb-df343bbf7230,2023-02-01T05:18:06.021Z,2023-02-01T05:18:06.021Z,torsionAngleDeviation,1;2;torsionAngleDeviation,False,None,1f6fc108-1978-485b-9b27-6fc440b3e087,"i18n{""en"": ""Angular degree"", ""ru"": ""Градус угл...",False,81b62c00-e2a3-490d-84b9-d239aa567389,SKN&Meteo_Aikhal_test,False,False,False,4.294967e+09,Градус угловой
7,33a268a8-2959-4bf0-b08d-5fdbaeafaff7,2023-02-01T05:18:06.021Z,2023-02-01T05:18:06.021Z,tiltAngle,1;2;tiltAngle,False,None,1f6fc108-1978-485b-9b27-6fc440b3e087,"i18n{""en"": ""Angular degree"", ""ru"": ""Градус угл...",False,81b62c00-e2a3-490d-84b9-d239aa567389,SKN&Meteo_Aikhal_test,False,False,False,4.294967e+09,Градус угловой
8,6d55994a-c130-4d86-b347-5fc95d5316e4,2023-02-01T05:18:06.020Z,2023-02-01T05:18:06.020Z,wireTemperature,1;2;wireTemperature,False,None,1f6fc108-1978-485b-9b27-6fc440b3e084,"i18n{""en"": ""Celsius"", ""ru"": ""Цельсий"", ""es"": ""...",False,81b62c00-e2a3-490d-84b9-d239aa567389,SKN&Meteo_Aikhal_test,False,False,False,4.294967e+09,Цельсий
9,a793167d-f1e9-453d-962b-4431d951fc8c,2023-02-01T05:18:06.019Z,2023-02-01T05:18:06.019Z,tiltAngleDeviation,1;2;tiltAngleDeviation,False,None,1f6fc108-1978-485b-9b27-6fc440b3e087,"i18n{""en"": ""Angular degree"", ""ru"": ""Градус угл...",False,81b62c00-e2a3-490d-84b9-d239aa567389,SKN&Meteo_Aikhal_test,False,False,False,4.294967e+09,Градус угловой


In [62]:
df_devices, devices_names_dict = natlogger.list_of_devices()

OK


In [63]:
df_devices.head()

,id,created,changed,name,typeId,typeName,teamId,teamName,lat,lng,imei,password,active,publicAvailable,altitude,timezoneId,timezoneOffset,locationManual,online,deviceData,phoneNumber,deviceTypeDescription,parameterMappings,deviceFields,parameterMappingsToDelete,compoundFields,timezoneLastUpdateDt,locationLastUpdateDt,lastDataReceivedDt,notConnectedNotificationDt
0,84fe76f8-5ac5-4e83-9e44-7d463a1b868b,2023-01-31T16:01:49.826Z,2023-02-09T07:38:57.079Z,SKN_Aikhal(Sokol-M2),b02eae33-6d98-41ec-a9d2-63905277a358,SKN,81b62c00-e2a3-490d-84b9-d239aa567389,SKN&Meteo_Aikhal_test,65.94965,111.517448,861937065188865,2211,True,True,635,None,None,True,True,"{'date': None, 'parameters': [{'name': 'Temper...",None,None,[],None,None,None,None,2023-03-07T09:45:33.000Z,2023-03-07T09:46:11.735Z,None


In [64]:
'''
EVS - накопленные события и ошибки
photo - фото

Газоанализаторы

CO
PM2.5 - ???
HCN
PM10 - ???
SO2
NO2
Dosimeter не подключен
NO
CO2
CH2O - на самом деле датчик влажности
NH3 - на самом деле датчик температуры
H2S

Метеостанции

RNL - уровень заполнения
RNC - интенсивность
t - температура
HM - относительная влажность
WD - направление ветра
PR - давление
WM - порыв ветра
WV - скорость ветра
'''

required_parameters_names = ['Temperature',
                             'Wind direction',
                             'Relative humidity',
                             'Pressure',
                             
                             '1;2;tiltAngle', 
                             '1;2;tiltAngleDeviation', 
                             '1;2;wireTemperature', 
                             '1;2;ctState',
                             '1;2;batteryVoltage', 
                             '1;2;torsionAngle',
                             '1;2;torsionAngleDeviation',
                             
                             '1;1;tiltAngle', 
                             '1;1;tiltAngleDeviation', 
                             '1;1;wireTemperature', 
                             '1;1;ctState',
                             '1;1;batteryVoltage', 
                             '1;1;torsionAngle',
                             '1;1;torsionAngleDeviation',
                            
                             '1;0;tiltAngle', 
                             '1;0;tiltAngleDeviation', 
                             '1;0;wireTemperature', 
                             '1;0;ctState',
                             '1;0;batteryVoltage', 
                             '1;0;torsionAngle',
                             '1;0;torsionAngleDeviation',
                             
                             '0;0;tiltAngle', 
                             '0;0;tiltAngleDeviation', 
                             '0;0;wireTemperature', 
                             '0;0;ctState',
                             '0;0;batteryVoltage', 
                             '0;0;torsionAngle',
                             '0;0;torsionAngleDeviation',
                            
                             '0;1;tiltAngle', 
                             '0;1;tiltAngleDeviation', 
                             '0;1;wireTemperature', 
                             '0;1;ctState',
                             '0;1;batteryVoltage', 
                             '0;1;torsionAngle',
                             '0;1;torsionAngleDeviation',
                            
                             '0;2;tiltAngle', 
                             '0;2;tiltAngleDeviation', 
                             '0;2;wireTemperature', 
                             '0;2;ctState',
                             '0;2;batteryVoltage', 
                             '0;2;torsionAngle',
                             '0;2;torsionAngleDeviation',]

In [65]:
'''
YYYY, MM, DD, h, m, s
'''

start_date = datetime.datetime(2023, 2, 14, 0, 0, 0, tzinfo=datetime.timezone.utc) 
end_date = datetime.datetime(2023, 3, 7, 0, 0, 0, tzinfo=datetime.timezone.utc)

In [66]:
'''
NONE - Без группировки
SECOND - Секунда
MINUTE - Минута
HOUR - Час
DAY - День
WEEK - Неделя
'''

df = natlogger.get_data(required_device_name='SKN_Aikhal(Sokol-M2)', 
                        devices_names_dict=devices_names_dict, 
                        required_parameters_names=required_parameters_names,
                        parameters_names_dict=parameters_names_dict,
                        start_date=start_date,
                        end_date=end_date,
                        group_type='MINUTE')

get data from  2023-02-14T00:00:00.000000Z  to  2023-03-07T00:00:00.000000Z
grouping to  MINUTE
device:  SKN_Aikhal(Sokol-M2)
required params:  ['Temperature', 'Wind direction', 'Relative humidity', 'Pressure', '1;2;tiltAngle', '1;2;tiltAngleDeviation', '1;2;wireTemperature', '1;2;ctState', '1;2;batteryVoltage', '1;2;torsionAngle', '1;2;torsionAngleDeviation', '1;1;tiltAngle', '1;1;tiltAngleDeviation', '1;1;wireTemperature', '1;1;ctState', '1;1;batteryVoltage', '1;1;torsionAngle', '1;1;torsionAngleDeviation', '1;0;tiltAngle', '1;0;tiltAngleDeviation', '1;0;wireTemperature', '1;0;ctState', '1;0;batteryVoltage', '1;0;torsionAngle', '1;0;torsionAngleDeviation', '0;0;tiltAngle', '0;0;tiltAngleDeviation', '0;0;wireTemperature', '0;0;ctState', '0;0;batteryVoltage', '0;0;torsionAngle', '0;0;torsionAngleDeviation', '0;1;tiltAngle', '0;1;tiltAngleDeviation', '0;1;wireTemperature', '0;1;ctState', '0;1;batteryVoltage', '0;1;torsionAngle', '0;1;torsionAngleDeviation', '0;2;tiltAngle', '0;2;tiltAng

In [67]:
df

,1;2;ctState,1;2;batteryVoltage,1;2;torsionAngleDeviation,1;2;tiltAngle,1;2;wireTemperature,1;2;tiltAngleDeviation,1;2;torsionAngle,1;1;torsionAngle,1;1;tiltAngle,1;1;tiltAngleDeviation,1;1;torsionAngleDeviation,1;1;ctState,1;1;wireTemperature,1;1;batteryVoltage,1;0;tiltAngleDeviation,1;0;tiltAngle,1;0;torsionAngle,1;0;wireTemperature,1;0;torsionAngleDeviation,1;0;ctState,1;0;batteryVoltage,0;2;tiltAngle,0;2;tiltAngleDeviation,0;2;ctState,0;2;torsionAngle,0;2;wireTemperature,0;2;torsionAngleDeviation,0;2;batteryVoltage,0;1;torsionAngleDeviation,0;1;torsionAngle,0;1;batteryVoltage,0;1;ctState,0;1;tiltAngleDeviation,0;1;wireTemperature,0;1;tiltAngle,0;0;tiltAngleDeviation,0;0;torsionAngleDeviation,0;0;torsionAngle,0;0;wireTemperature,0;0;batteryVoltage,0;0;tiltAngle,0;0;ctState,Pressure,Relative humidity,Wind direction,Temperature
2023-02-14T00:01:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,946.89,81.0,142.0,-29.74
2023-02-14T00:11:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,946.89,81.0,147.0,-29.60
2023-02-14T00:21:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,946.68,80.0,144.0,-29.82
2023-02-14T00:31:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,946.80,81.0,163.0,-29.28
2023-02-14T00:41:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,946.80,81.0,207.0,-29.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-06T23:35:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,933.39,84.0,196.0,-32.41
2023-03-06T23:36:00.000Z,1.0,4.2,6.0,-3.42,-27.6,7.0,-1.04,-3.50,-4.77,3.0,2.0,1.0,-27.7,4.1,29.0,3.73,-0.66,-27.9,31.0,1.0,4.1,2.38,21.0,1.0,-4.39,-27.3,26.0,3.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,32.0,-8.31,-27.0,4.2,0.40,1.0,NaN,NaN,NaN,NaN
2023-03-06T23:45:00.000Z,1.0,4.2,5.0,-3.43,-27.4,5.0,-1.03,-3.22,-4.74,7.0,6.0,1.0,-27.6,4.1,34.0,3.58,-0.97,-27.7,36.0,1.0,4.1,1.23,28.0,1.0,-4.45,-27.3,41.0,3.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,34.0,-5.75,-27.0,4.2,0.01,1.0,933.30,81.0,193.0,-32.49
2023-03-06T23:55:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,933.39,83.0,217.0,-30.99


In [68]:
natlogger.save_records_to_excel()

---